# Preprocessamento do DODF para segmentacao 

Separa sentencas e as rotula no padrao IOB (B=sentenca de inicio do ato, I=sentenca continuacao do ato, O=sentenca nao faz parte do ato)

In [1]:
import json
import pandas as pd
import glob
import re

## Parte 1 - Identificando os DODFs com anotacao de aposentadoria

In [2]:
df = pd.read_csv("labeled.csv")
df.columns

Index(['DATA_DODF', 'NUM_DODF', 'ATO', 'EMPRESA_ATO', 'COD_MATRICULA_ATO',
       'NOME_ATO', 'CARGO', 'CLASSE', 'PADRAO', 'QUADRO', 'PROCESSO',
       'FUND_LEGAL', 'text', 'labels'],
      dtype='object')

In [3]:
month = ["01_Janeiro/", "02_Fevereiro/", "03_Março/", "04_Abril/", "05_Maio/", "06_Junho/", "07_Julho/", "08_Agosto/", "09_Setembro/", "10_Outubro/", "11_Novembro/", "12_Dezembro/"]
dodfs = []
dic = {}

for i in range(len(df)):
    num_dodf = int(df.loc[i, 'NUM_DODF'])
    data_dodf= df.loc[i, 'DATA_DODF']
    if df.loc[i, 'text']:
        if (num_dodf, data_dodf) not in dic:
            dic[(num_dodf, data_dodf)] = -1
            dia, mes, ano = data_dodf.split('/')
            path = "../dodfs/results/json/" + ano + "/" + month[int(mes)-1]
            for i in glob.glob(path+'*.json'):
                aux = i.split('/')[-1]
                aux = aux.split()
                if int(aux[1])==num_dodf and aux[2] == data_dodf.replace('/', '-') and not re.search("SUPLEMENTO", ' '.join(aux)):
                    dic[(num_dodf, data_dodf)] = i
                    dodfs.append(i)

In [4]:
len(dodfs), len(dic)

(226, 227)

In [5]:
for i in dic:
    if dic[i] not in dodfs:
        print("Missing DODF:", i)

Missing DODF: (76, '20/04/2018')


In [6]:
dodfs.append("../dodfs/results/json/2018/04_Abril/DODF 076 20-04-2018 SUPLEMENTO.json")

## Parte 2 - Identificando trechos de texto contendo ato de aposentadoria nos blocos do DODF

In [7]:
path = "../dodfs/results/json/2018/04_Abril/DODF 019 04-04-2018 EDICAO EXTRA.json"
f = open(path)
data = json.load(f)

In [8]:
full_base = []
not_found = []
cnt = 0
temp = -1
for dodf in dodfs:
    f = open(dodf)
    data = json.load(f)
    aux = df.loc[(df['NUM_DODF'] == int(dodf.split('/')[-1].split()[1])) & (df['DATA_DODF'] == dodf.split('/')[-1].split()[2].replace('-', '/'))]
    for j in range(len(data)):
        data[j].append([])
    for i in aux.index:
        cnt += 1
        if not df.loc[i, 'text'] or pd.isna(df.loc[i, 'text']):
            continue
        find = False
        for j in range(len(data)):
            entity = re.search(df.loc[i, 'text'].replace('(', '\(').replace(')', '\)'), data[j][4].replace('\n', ' '))
            if entity:
                temp = entity
                find = True
                if entity.span() not in data[j][5]:
                    data[j][5].append(entity.span())
        if not find:
            not_found.append((dodf, df.loc[i, 'text'], i))
    full_base.append(data)


In [9]:
print(f"Total de anotacoes de atos:{cnt}\nAtos nao identificados:{len(not_found)}")

Total de anotacoes de atos:5516
Atos nao identificados:111


## Parte 3 - Salvando os dados rotulados padrao MUC

In [10]:
# Fix - full_base[137][316][5] tinha um trecho de texto que cobria dois atos, corrigido para nao dar overlap
# no segundo ato de aposentadoria
for i in range(len(full_base[137][316][5])):
    if full_base[137][316][5][i] == (5637, 6496):
        print("trocou")
        full_base[137][316][5][i] = (6147, 6496)

trocou


In [11]:
def MUCify_block(block, i, j):
    MUC_data = []
    texto = block[4].replace('\n', ' ')
    if block[5]:
        block[5].sort(reverse=True)
        for ato in block[5]:
            if re.search("</*aposentadoria>", texto[ato[0]:ato[1]+1]):
                print(f"DODF:{i}, bloco:{j}")
                print("Sobreposicao de atos")
                break
            texto = texto[:ato[0]] + "<aposentadoria>" + texto[ato[0]:ato[1]+1] + "</aposentadoria>" + texto[ato[1]+1:]
    return texto

# aux = full_base[0][364]
# aux_r = MUCify_sentences(aux)

In [12]:
muc_base = []
# Para cada DODF na base
for i in range(len(full_base)):
    # Para cada bloco no DODF
    for j in range(len(full_base[i])):
#         print(f"DODF:{i}, bloco:{j}")
        muc_base.append(MUCify_block(full_base[i][j], i, j))
#         full_base[i][j][4] =  MUCify_block(full_base[i][j], i, j)

In [ ]:
# def MUCtoIOB_block(block):
#     labels = []
#     texto = block.split()
#     i = 0
#     while i < len(texto):
        